# Procesamiento de Datos por Lotes desde una API

En este laboratorio, aprenderás cómo interactuar con la API de Spotify y extraer datos de la API de manera por lotes. Explorarás qué significa la paginación y cómo enviar una solicitud a la API que requiere autorización.


# Tabla de Contenidos

- [ 1 - Crear una APP de Spotify](#1)
- [ 2 - Entender los Conceptos Básicos de las APIs](#2)
  - [ 2.1 - Obtener Token](#2-1)
  - [ 2.2 - Obtener Nuevos Lanzamientos](#2-2)
    - [ Ejercicio 1](#ex01)
  - [ 2.3 - Paginación](#2-3)
    - [ Ejercicio 2](#ex02)
    - [ Ejercicio 3](#ex03)
  - [ 2.4 - Opcional - Límites de Tasa de la API](#2-4)
- [ 3 - Pipeline por lotes](#3)
  - [ Ejercicio 4](#ex04)
  - [ Ejercicio 5](#ex05)
  - [ Ejercicio 6](#ex06)
- [ 4 - Opcional - SDK Spotipy](#4)
  - [ Ejercicio 7](#ex07)


<a id='1'></a>
## 1 - Crear una APP de Spotify

Para obtener acceso a los recursos de la API, necesitas crear una cuenta de Spotify si aún no tienes una. Una cuenta de prueba será suficiente para completar este laboratorio.

1. Ve a https://developer.spotify.com/, crea una cuenta e inicia sesión.
2. Haz clic en el nombre de la cuenta en la esquina superior derecha y luego haz clic en **Dashboard**.
3. Crea una nueva APP usando los siguientes detalles:
   - Nombre de la APP: `dec2w2a1-spotify-app`
   - Descripción de la APP: `aplicación de spotify para probar la API`
   - Sitio web: dejar en blanco
   - URIs de redirección: `http://localhost:3000`
   - API a usar: selecciona `Web API`
4. Haz clic en el botón **Save**. Si recibes un mensaje de error diciendo que tu cuenta no está lista, puedes cerrar sesión, esperar unos minutos y luego repetir los pasos 2-4.
5. En la página de inicio de la APP haz clic en **Settings** y revela `Client ID` y `Client secret`. Guárdalos en el archivo `src/env` proporcionado en este laboratorio. Asegúrate de guardar el archivo `src/env` usando `Ctrl + S` o `Cmd + S`.

Aquí está el enlace a [la documentación de la API de Spotify](https://developer.spotify.com/documentation/web-api/tutorials/getting-started) a la que puedes referirte mientras trabajas en los ejercicios del laboratorio. La información requerida para completar las tareas será proporcionada durante el laboratorio. Interactuarás con dos recursos:
- Nuevos lanzamientos de álbumes en la primera y segunda parte ([endpoint](https://developer.spotify.com/documentation/web-api/reference/get-new-releases));
- Pistas de álbum en la segunda parte ([endpoint](https://developer.spotify.com/documentation/web-api/reference/get-an-albums-tracks)).


<a id='2'></a>
## 2 - Entender los conceptos básicos de las APIs

Varios paquetes en Python permiten solicitar datos de una API; en este laboratorio, usarás el paquete `requests`, que es una biblioteca popular y versátil para realizar solicitudes HTTP. Proporciona una forma sencilla y fácil de usar para interactuar con servicios web y APIs. Carguemos los paquetes necesarios:


In [1]:
import os
from typing import Dict, Any, Callable

from dotenv import load_dotenv
import json
import requests 

<a id='2-1'></a>
### 2.1 - Obtener Token

El primer paso al trabajar con una API es entender el proceso de autenticación. Para ello, la APP de Spotify genera un ID de cliente y un secreto de cliente que usarás para generar un token de acceso. El token de acceso es una cadena que contiene las credenciales y permisos que puedes usar para acceder a un recurso dado. Puedes encontrar más información al respecto en la [documentación de la API](https://developer.spotify.com/documentation/web-api/concepts/access-token). Dado que cada API se desarrolla con un propósito particular, es necesario que siempre leas y comprendas las particularidades de cada API para que puedas acceder a los datos de manera responsable. A lo largo de este laboratorio, se te proporcionarán varios enlaces a la documentación y se te anima a que los leas. (Durante la sesión de laboratorio, puedes pasar rápidamente por los enlaces, pero siempre puedes revisarlos con más detalle después de la sesión de laboratorio).

Creemos algunas variables para contener los valores del client_id y el client_secret que almacenaste en el archivo src/env.


In [2]:
load_dotenv('./src/env', override=True)

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

La función `get_token` a continuación toma un ID de cliente, un secreto de cliente y una URL como entrada, y realiza una solicitud POST a esa URL para obtener un token de acceso usando las credenciales del cliente. Ejecute la siguiente celda para obtener el token de acceso.


In [3]:
def get_token(client_id: str, client_secret: str, url: str) -> Dict[Any, Any]:
    """Permite realizar una solicitud POST para obtener un token de acceso.
    
    Args:
        client_id (str): ID de cliente de la aplicación.
        client_secret (str): Secreto de cliente de la aplicación.
        url (str): URL para realizar la solicitud POST.
        
    Returns:
        Dict[Any, Any]: Diccionario que contiene el token de acceso.
    """
    # Definimos los encabezados para la solicitud, indicando que el 
    # contenido es de tipo formulario
        
    headers = {        
        "Content-Type": "application/x-www-form-urlencoded"            
    }
    
    payload = { # Tipo de concesión que estamos utilizando
                "grant_type"    : "client_credentials", 
                # ID de cliente proporcionado
                "client_id"     : client_id, 
                # Secreto de cliente proporcionado
                "client_secret" : client_secret
               }
    
    try: 
         # Realizamos la solicitud POST a la URL especificada
        response = requests.post( 
                                 url     = url, 
                                 headers = headers, 
                                 data    = payload
                                 )
        print(type(response))
        # Verificamos si la respuesta fue exitosa, si no, se lanzará una excepción
        response.raise_for_status()
        # Convertimos la respuesta JSON en un diccionario de Python
        response_json = json.loads(response.content)
        
        return response_json
        
    except Exception as err:
        print(f"Error: {err}")
        return {}

URL_TOKEN = "https://accounts.spotify.com/api/token"

token = get_token(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, url=URL_TOKEN)

print(token)

<class 'requests.models.Response'>
{'access_token': 'BQAboBQZHtRuaOpaRc1I6POh6tLKUsq8RtyRbETuVyLR1cT9SAg_PpQ6MfkwlkXZCAOoDAu7iNEBW2ZoONvIFx4zX4RJ1arkDvaBJCE-Cza5tBQ67GkdEGdWBEB1akD33QVYQtBZ5dg', 'token_type': 'Bearer', 'expires_in': 3600}


In [4]:
print(json.dumps(token, indent=4))

{
    "access_token": "BQAboBQZHtRuaOpaRc1I6POh6tLKUsq8RtyRbETuVyLR1cT9SAg_PpQ6MfkwlkXZCAOoDAu7iNEBW2ZoONvIFx4zX4RJ1arkDvaBJCE-Cza5tBQ67GkdEGdWBEB1akD33QVYQtBZ5dg",
    "token_type": "Bearer",
    "expires_in": 3600
}


Puedes ver que se te proporciona un token de acceso temporal. El campo `expires_in` te indica la duración de este token en segundos. Cuando este token expire, tus solicitudes fallarán y se te devolverá un objeto de error con un código de estado de 401. Este código de estado significa que la solicitud no está autorizada.

Siempre que envíes una solicitud a la API de spotify, necesitas incluir en la solicitud el token de acceso, como un encabezado de autorización siguiendo un formato determinado. Se te proporciona la función `get_auth_header`. Esta función espera el token de acceso y devuelve el encabezado de autorización que puede incluirse en la solicitud de la API.

Asegúrate de ejecutar la siguiente celda para declarar la función `get_auth_header`, que usarás a lo largo de este laboratorio.


In [5]:
def get_auth_header(access_token: str) -> Dict[str, str]:
    return {"Authorization": f"Bearer {access_token}"}

Ahora, usemos el token para realizar una solicitud para acceder al primer recurso, que es el [nuevos lanzamientos](https://developer.spotify.com/documentation/web-api/reference/get-new-releases).


<a id='2-2'></a>
### 2.2 - Obtener nuevos lanzamientos


<a id='ex01'></a>
### Ejercicio 1

Sigue las instrucciones para completar la función `get_new_releases`:

1. Llama a la función `get_auth_header` y pásale el token de acceso (que se especifica como entrada a la función `get_new_releases`). Guarda la salida de `get_auth_header` en una variable llamada `headers`.
2. Se te proporciona la URL en la variable `request_url`. Usa esta URL y el encabezado del paso anterior para realizar una solicitud `get()`.
3. La solicitud `response` es un objeto de tipo `requests.models.Response`. Este objeto tiene un método llamado `json()` que te permite transformar el contenido de la respuesta en un objeto JSON o en un diccionario de Python simple. Usa este método en el objeto `response` para devolver el contenido como un diccionario de Python.

Luego usarás la URL o endpoint proporcionado `URL_NEW_RELEASES` para realizar llamadas a la API, pasando el valor de `access_token` del objeto `token` que obtuviste antes.


In [6]:
def get_new_releases(url: str, access_token: str, offset: int=0, limit: int=20, next: str="") -> Dict[Any, Any]:
    """Realiza una solicitud GET al endpoint de nuevos lanzamientos.
    
    Args:
        url (str): URL base para la solicitud.
        access_token (str): Token de acceso.
        offset (int, optional): Desplazamiento de página para la paginación. Por defecto es 0.
        limit (int, optional): Número de elementos por página. Por defecto es 20.
        next (str, optional): URL para realizar la siguiente solicitud. Por defecto es "".
        
    Returns:
        Dict[Any, Any]: Respuesta de la solicitud.
    """

    if next == "":        
        request_url = f"{url}?offset={offset}&limit={limit}"
    else: 
        request_url = f"{next}"

    ### START CODE HERE ### (~ 4 lines of code)
    # Call get_auth_header() function and pass the access token.
    # headers = None(access_token=None)
    
    # Llamamos a la función get_auth_header() y pasamos el token de acceso.
    headers = get_auth_header(access_token=access_token)
    
    
    try: 
        # Perform a get() request using the request_url and headers.
        # response = requests.None(url=None, headers=None)
        
        # Realizamos una solicitud GET utilizando request_url y headers.
        response = requests.get(url=request_url, headers=headers)
        
        
        # Use json() method over the response to return it as Python dictionary.
         # Usamos el método json() sobre la respuesta 
         # para devolverlo como un diccionario de Python.
        return response.json()
    ### END CODE HERE ###
    
    except Exception as err:
        print(f"Error requesting data: {err}")
        print(f"Error al solicitar datos: {err}")
        return {'error': err}
        
URL_NEW_RELEASES = "https://api.spotify.com/v1/browse/new-releases"

# Note: the `access_token` value from the dictionary `token` can be retrieved either using `get()` method or dictionary syntax `token['access_token']`
# Nota: el valor de `access_token` del diccionario `token` 
# se puede recuperar utilizando el método `get()` 
# o la sintaxis de diccionario `token['access_token']`
releases_response = get_new_releases(
                url = URL_NEW_RELEASES, 
                access_token = token.get('access_token'))


In [7]:

print(json.dumps(releases_response, indent=4))

{
    "albums": {
        "href": "https://api.spotify.com/v1/browse/new-releases?offset=0&limit=20",
        "items": [
            {
                "album_type": "album",
                "artists": [
                    {
                        "external_urls": {
                            "spotify": "https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02"
                        },
                        "href": "https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02",
                        "id": "06HL4z0CvFAxyc27GXpf02",
                        "name": "Taylor Swift",
                        "type": "artist",
                        "uri": "spotify:artist:06HL4z0CvFAxyc27GXpf02"
                    }
                ],
                "available_markets": [
                    "AR",
                    "AU",
                    "AT",
                    "BE",
                    "BO",
                    "BR",
                    "BG",
                    "CA",
           

El resultado que obtienes es un objeto JSON que fue transformado en un diccionario de Python. Puedes explorar la estructura de la respuesta que recibes:


In [8]:
releases_response.keys()

dict_keys(['albums'])

In [9]:
releases_response.get('albums').keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [10]:
releases_response.get('albums').get('offset')

0

In [11]:
releases_response.get('albums').get('limit')

20

In [12]:
releases_response.get('albums').get('next')

'https://api.spotify.com/v1/browse/new-releases?offset=20&limit=20'

Cada API gestiona las respuestas de su propia manera, por lo que se recomienda encarecidamente leer la documentación y entender las diferencias sutiles detrás de los puntos finales de la API con los que estás trabajando. En este caso, ves algunos campos como `'href'` bajo el campo `'albums'`, que te indica la URL utilizada para la solicitud que acabas de enviar.


In [13]:
releases_response.get('albums').get('total')

100

Puedes ver que hay dos parámetros: `offset` y `limit` que fueron agregados al endpoint. Esos parámetros son la base de la paginación en este endpoint de API. Los revisaremos más tarde.

También puedes explorar los elementos devueltos usando el campo `'items'` bajo `'albums'`. Esto devolverá una lista de elementos, puedes echar un vistazo al número de elementos devueltos:


In [14]:
len(releases_response.get('albums').get('items'))

20

Explore los artículos:


In [15]:
releases_response.get('albums').get('items')[0]

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
   'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
   'id': '06HL4z0CvFAxyc27GXpf02',
   'name': 'Taylor Swift',
   'type': 'artist',
   'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}],
 'available_markets': ['AR',
  'AU',
  'AT',
  'BE',
  'BO',
  'BR',
  'BG',
  'CA',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DK',
  'DO',
  'DE',
  'EC',
  'EE',
  'SV',
  'FI',
  'FR',
  'GR',
  'GT',
  'HN',
  'HK',
  'HU',
  'IS',
  'IE',
  'IT',
  'LV',
  'LT',
  'LU',
  'MY',
  'MT',
  'MX',
  'NL',
  'NZ',
  'NI',
  'NO',
  'PA',
  'PY',
  'PE',
  'PH',
  'PL',
  'PT',
  'SG',
  'SK',
  'ES',
  'SE',
  'CH',
  'TW',
  'TR',
  'UY',
  'US',
  'GB',
  'AD',
  'LI',
  'MC',
  'ID',
  'JP',
  'TH',
  'VN',
  'RO',
  'IL',
  'ZA',
  'SA',
  'AE',
  'BH',
  'QA',
  'OM',
  'KW',
  'EG',
  'MA',
  'DZ',
  'TN',
  'LB',
  'JO',
  'PS',
  'IN',
  'KZ',
  'MD

<a id='2-3'></a>
### 2.3 - Paginación

Si imprimes `releases_response`, puedes ver los siguientes campos:

```json
{
...,
'limit': 20,
'next': 'https://api.spotify.com/v1/browse/new-releases?offset=20&limit=20',
'offset': 0,
'previous': None,
'total': 100
}
```

Aunque hay un total de 100 elementos disponibles para ser devueltos, solo se devolvieron 20. Esto lo establece el parámetro `limit` y esos fueron los 20 elementos que acabas de contar antes. Este límite en la cantidad de elementos devueltos es una característica común de varias APIs y aunque en algunos casos puedes modificar dicho límite, una buena práctica es usarlo con **paginación** para obtener todos los elementos que se pueden devolver.

Cada API maneja la paginación de manera diferente. Para Spotify, la respuesta de las solicitudes te proporciona dos campos que te permiten consultar las diferentes páginas de tu solicitud: `previous` y `next`. Estos dos campos devolverán la URL a la página anterior o siguiente respectivamente y se basan en los parámetros `offset` y `limit`. En este caso, hay dos formas de explorar el resto de los datos:

- puedes usar el valor del parámetro `next` para obtener la URL directa para la siguiente página de solicitudes, o
- puedes construir la URL para la siguiente página desde cero usando los parámetros `offset` y `limit` (asegúrate de actualizar el parámetro `offset` para la solicitud).

Para fines de aprendizaje, usarás el método 2 para construir la URL tú mismo. Luego también lo compararás con el resultado de usar el primer método solo para verificar que creaste la URL correctamente.

Antes de crear una función que te permita paginar, intentemos hacerlo manualmente. Si comparas las URLs proporcionadas por los campos `href` y `next`, puedes ver que mientras el parámetro `limit` permanece igual, el parámetro `offset` ha aumentado con el mismo valor que el almacenado en `limit`.

```json
{
...,
'href': 'https://api.spotify.com/v1/browse/new-releases?offset=0&limit=20',
...,
'next': 'https://api.spotify.com/v1/browse/new-releases?offset=20&limit=20',
...
}
```

Así que para nuestra próxima llamada, pasemos 20 a `offset` y mantengamos `limit` en 20:


In [16]:
next_releases_response = get_new_releases(url=URL_NEW_RELEASES, access_token=token.get('access_token'), offset=20, limit=20)

Verifique los valores para `href` y `next` en la nueva respuesta `next_releases_response`:


In [17]:
next_releases_response.get('albums').get('href')

'https://api.spotify.com/v1/browse/new-releases?offset=20&limit=20'

In [18]:
next_releases_response.get('albums').get('next')

'https://api.spotify.com/v1/browse/new-releases?offset=40&limit=20'

Dado estos resultados, puedes ver que el `offset` aumenta en el valor del `limit`. Como las respuestas muestran que el valor de `total` es 100, esto significa que puedes acceder a la última página de respuestas usando un `offset` de 80, mientras mantienes el valor de `limit` en 20.


In [19]:
last_releases_response = get_new_releases(url=URL_NEW_RELEASES, access_token=token.get('access_token'), offset=80, limit=20)

In [20]:
print(last_releases_response.get('albums').get('previous'))
print(last_releases_response.get('albums').get('next'))

https://api.spotify.com/v1/browse/new-releases?offset=60&limit=20
None


Puedes ver que el valor del campo `next` es `None`, indicando que llegaste a la última página. Por otro lado, puedes ver que `previous` contiene la URL para solicitar los datos de la página anterior, así que incluso puedes retroceder si es necesario.


<a id='ex02'></a>
### Ejercicio 2

Sigue las instrucciones para crear una nueva función que manejará la paginación, basada en la función `get_new_releases`:

1. Revisa la definición de la función, debes proporcionar un callable (`endpoint_request`) que corresponda a la función que realiza la llamada a la API para obtener los lanzamientos de nuevos álbumes.
2. Antes del bucle `while`, crea un diccionario llamado `kwargs` con las siguientes claves:
    * `'url'`: la URL para realizar la llamada pasada a la función como parámetro.
    * `'access_token'`: el token de acceso pasado a la función como parámetro.
    * `'offset'`: desplazamiento de la página para la solicitud paginada.
    * `'limit'`: número máximo de elementos en la solicitud de la página.
3. Llama a la función `endpoint_request()` con los argumentos clave que especificaste en el diccionario `kwargs`. Asignarlo a `response`.
4. Extiende la lista `responses` con los `items` del álbum de la `response`.
5. Crea una variable `total_elements` que contenga el número total de elementos de la `response`. Recuerda que la `response` tiene un campo llamado `'albums'` que tiene el número `'total'` de elementos. Si tienes alguna duda sobre la estructura de la respuesta, recuerda consultar la [documentación](https://developer.spotify.com/documentation/web-api/reference/get-new-releases).
6. Ejecuta el bucle `while` mientras que el valor de `offset` sea menor que la variable `total_elements` que definiste antes.
7. Dentro del bucle `while` realiza los siguientes pasos:
   * Actualiza el valor de `offset` con el valor actual de la solicitud que hiciste más el valor de `limit`.
   * Repite la definición del diccionario `kwargs` con los mismos parámetros. Ten en cuenta que en este caso el valor de `offset` ha sido actualizado.
   * Repite los pasos 3 y 4.


In [21]:
def paginated_new_releases(endpoint_request: Callable, 
                           url: str, 
                           access_token: str, 
                           offset: int=0, 
                           limit: int=20) -> list:
    """Permite realizar la paginación sobre una solicitud a la API realizada por la función endpoint_request.
    
    Args:
        endpoint_request (Callable): Función que realiza las llamadas a la API.
        url (str): URL del endpoint para la solicitud.
        access_token (str): Token de acceso.
        offset (int, optional): Desplazamiento de la solicitud de la página. Por defecto es 0.
        limit (int, optional): Límite de la solicitud de la página. Por defecto es 20.
        
    Returns:
        list: Lista con los elementos solicitados.
    """
    
    responses = [] # Lista para almacenar las respuestas
    
    ### START CODE HERE ### (~ 19 lines of code)
    # Create a dictionary named kwargs with the values corresponding to the keys url, token, offset, limit
    
    # kwargs = {         
    #         "url"          : None,
    #         "access_token" : None,
    #         "offset"       : None,
    #         "limit"        : None,
    #     } 
    
    kwargs = {         
        "url"          : url,
        "access_token" : access_token,
        "offset"       : offset,
        "limit"        : limit,
    } 
    
    # Call the endpoint_request() function with the arguments specified in the kwargs dictionary.
    # response = None(**None)
    
    # Llamamos a la función endpoint_request() 
    # con los argumentos especificados en el diccionario kwargs.
    response = endpoint_request(**kwargs)
    
    # Use extend() method to add the albums' items to the list of responses.
    # responses.None(response.None('None').None('None'))
    
    # Usamos el método extend() para agregar los ítems de álbumes 
    # a la lista de respuestas.
    responses.extend(response['albums']['items'])
    
    # Get the total number of the elements in albums and save it in the variable total_elements.
    # total_elements = response.None('None').None('None')

    # Obtenemos el número total de elementos en albums y lo guardamos en la variable total_elements.
    total_elements = response['albums']['total']

    # Run the loop as long as the offset value is smaller than total_elements.
    # while None < None:
    
    # Ejecutamos el bucle mientras el valor de offset sea menor que total_elements.
    while offset < total_elements:
    
        # Update the offset value with the current value from the request you did plus the limit value.
        # offset = response.None('None').None('None') + None
        
        offset = response['albums']['offset'] + limit
        
        # Repeat the definition of the kwargs dictionary with the same parameters (with the new offset value).
        # kwargs = {             
        #     "url": None,
        #     "access_token": None,
        #     "offset": None,
        #     "limit": None,
        # }        
        
         # Repetimos la definición del diccionario kwargs con los mismos parámetros (con el nuevo valor de offset).
        kwargs = {             
            "url"         : url,
            "access_token": access_token,
            "offset"      : offset,
            "limit"       : limit,
        }      
        
         
        # Call the endpoint_request() function with the arguments specified in the kwargs dictionary.
        # response = None(**None)
        
        # Llamamos a la función endpoint_request() con los argumentos especificados en el diccionario kwargs.
        response = endpoint_request(**kwargs)
        
        
        # Use extend() method to add the albums' items to the list of responses.
        # responses.None(response.None('None').None('None'))
        
        # Usamos el método extend() para agregar los ítems de álbumes a la lista de respuestas.
        responses.extend(response['albums']['items'])
        ### END CODE HERE ###
        
        print(f"Finished iteration for page with offset: {offset-limit}")

    return responses

Ahora, ejecuta el `paginated_new_releases` con la función `get_new_releases` como el parámetro llamable `endpoint_request`. Usa la misma URL utilizada en la llamada anterior a `get_new_releases`, así como el token de acceso. Establece el `offset` inicial en 0. Para el límite, el valor predeterminado es 20, pero puedes jugar con otros valores si quieres.


In [22]:
responses = paginated_new_releases(endpoint_request=get_new_releases,
                                   url=URL_NEW_RELEASES, 
                                   access_token=token.get('access_token'), 
                                   offset=0, limit=20)

Finished iteration for page with offset: 0
Finished iteration for page with offset: 20
Finished iteration for page with offset: 40
Finished iteration for page with offset: 60
Finished iteration for page with offset: 80


##### __Salida Esperada__ 
```text
Finalizó la iteración para la página con desplazamiento: 0
Finalizó la iteración para la página con desplazamiento: 20
Finalizó la iteración para la página con desplazamiento: 40
Finalizó la iteración para la página con desplazamiento: 60
Finalizó la iteración para la página con desplazamiento: 80
```


Echa un vistazo a uno de los ítems:


In [23]:
responses[0]

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
   'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
   'id': '06HL4z0CvFAxyc27GXpf02',
   'name': 'Taylor Swift',
   'type': 'artist',
   'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}],
 'available_markets': ['AR',
  'AU',
  'AT',
  'BE',
  'BO',
  'BR',
  'BG',
  'CA',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DK',
  'DO',
  'DE',
  'EC',
  'EE',
  'SV',
  'FI',
  'FR',
  'GR',
  'GT',
  'HN',
  'HK',
  'HU',
  'IS',
  'IE',
  'IT',
  'LV',
  'LT',
  'LU',
  'MY',
  'MT',
  'MX',
  'NL',
  'NZ',
  'NI',
  'NO',
  'PA',
  'PY',
  'PE',
  'PH',
  'PL',
  'PT',
  'SG',
  'SK',
  'ES',
  'SE',
  'CH',
  'TW',
  'TR',
  'UY',
  'US',
  'GB',
  'AD',
  'LI',
  'MC',
  'ID',
  'JP',
  'TH',
  'VN',
  'RO',
  'IL',
  'ZA',
  'SA',
  'AE',
  'BH',
  'QA',
  'OM',
  'KW',
  'EG',
  'MA',
  'DZ',
  'TN',
  'LB',
  'JO',
  'PS',
  'IN',
  'KZ',
  'MD

Puedes verificar la variable `responses` para ver si todos los elementos se descargaron con éxito.


In [24]:
len(responses)

100

Con la función `paginated_new_releases` que creaste, ahora puedes obtener los 100 artículos disponibles.


<a id='ex03'></a>
### Ejercicio 3

La función `get_new_releases` puede manejar la paginación pasando los parámetros `offset` y `limit` o solo usando el parámetro `next`. Crea otra función que use el parámetro `next` para realizar la paginación y compara tus resultados con el ejercicio anterior. Sigue las instrucciones a continuación:

El diccionario `kwargs` ahora está definido con las siguientes claves:
- `'url'`: la URL para realizar la llamada pasada a la función como parámetro.
- `'access_token'`: el token de acceso pasado a la función como parámetro.
- `'next'`: la URL para generar la siguiente solicitud, definida como una cadena vacía para la primera llamada.

Dentro del `while`:
1. Llama a la función `endpoint_request()` con los argumentos de palabras clave que especificaste en el diccionario `kwargs`. Asigna esto a `response`.
2. Extiende la lista `responses` con los `items` de los álbumes de `response`.
3. Reasigna el valor de `next_page` como el valor `'next'` del diccionario `response["albums"]`. Si tienes alguna duda sobre la estructura de la respuesta, recuerda consultar la [documentación](https://developer.spotify.com/documentation/web-api/reference/get-new-releases).
4. Actualiza el diccionario `kwargs`: establece el valor de la clave `'next'` como la variable `next_page`.


In [25]:
def paginated_with_next_new_releases(endpoint_request: Callable, 
                                     url: str, 
                                     access_token: str) -> list:
    """Gestiona la paginación para solicitudes a la API realizadas con la función endpoint_request.
    
    Args:
        endpoint_request (Callable): Función que realiza la solicitud a la API.
        url (str): URL base para la solicitud.
        access_token (str): Token de acceso.
        
    Returns:
        list: Respuestas almacenadas en una lista.
    """
    # Lista para almacenar todas las respuestas
    responses = []
    
    # Inicializamos next_page con la URL base
    next_page = url
    
    kwargs = {
            "url"          : url,
            "access_token" : access_token,
            "next"         : ""
        }
    
    while next_page:
        
        ### START CODE HERE ### (~ 4 lines of code)
        # Call the endpoint_request() function with the arguments specified in the kwargs dictionary.
        # response = None(**None)
        
        # Llamamos a la función endpoint_request() con los 
        # argumentos especificados en el diccionario kwargs.
        response = endpoint_request(**kwargs)  
        
        # Use extend() method to add the albums' items to the list of responses.
        # responses.None(response.None('None').None('None'))
        
        # Usamos el método extend() para agregar los elementos 
        # de los álbumes a la lista de respuestas.
        responses.extend(response['albums']['items'])
        
        
        # Reassign the value of next_page as the 'next' value from the response["albums"] dictionary.
        # next_page = response.None('None').None('None')
        
        # Reasignamos el valor de next_page como el valor 'next' 
        # del diccionario response["albums"].
        next_page = response['albums'].get('next')
        
        
        # Update the kwargs dictionary: set the value of the key 'next' as the variable next_page.
        # kwargs["None"] = None
        kwargs['next'] = next_page
        # Actualizamos el diccionario kwargs: establecemos el valor de la clave 'next' como la variable next_page.
        
        
        ### END CODE HERE ###
        
        print(f"Executed request with URL: {response.get('albums').get('href')}.")
                
    return responses
    

Ahora, realiza la nueva llamada paginada:


In [26]:
responses_with_next = paginated_with_next_new_releases(endpoint_request=get_new_releases, 
                                                             url=URL_NEW_RELEASES, 
                                                             access_token=token.get('access_token'))

Executed request with URL: https://api.spotify.com/v1/browse/new-releases?offset=0&limit=20.
Executed request with URL: https://api.spotify.com/v1/browse/new-releases?offset=20&limit=20.
Executed request with URL: https://api.spotify.com/v1/browse/new-releases?offset=40&limit=20.
Executed request with URL: https://api.spotify.com/v1/browse/new-releases?offset=60&limit=20.
Executed request with URL: https://api.spotify.com/v1/browse/new-releases?offset=80&limit=20.


Echa un vistazo a una de las respuestas:


In [27]:
responses_with_next[0]

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
   'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
   'id': '06HL4z0CvFAxyc27GXpf02',
   'name': 'Taylor Swift',
   'type': 'artist',
   'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}],
 'available_markets': ['AR',
  'AU',
  'AT',
  'BE',
  'BO',
  'BR',
  'BG',
  'CA',
  'CL',
  'CO',
  'CR',
  'CY',
  'CZ',
  'DK',
  'DO',
  'DE',
  'EC',
  'EE',
  'SV',
  'FI',
  'FR',
  'GR',
  'GT',
  'HN',
  'HK',
  'HU',
  'IS',
  'IE',
  'IT',
  'LV',
  'LT',
  'LU',
  'MY',
  'MT',
  'MX',
  'NL',
  'NZ',
  'NI',
  'NO',
  'PA',
  'PY',
  'PE',
  'PH',
  'PL',
  'PT',
  'SG',
  'SK',
  'ES',
  'SE',
  'CH',
  'TW',
  'TR',
  'UY',
  'US',
  'GB',
  'AD',
  'LI',
  'MC',
  'ID',
  'JP',
  'TH',
  'VN',
  'RO',
  'IL',
  'ZA',
  'SA',
  'AE',
  'BH',
  'QA',
  'OM',
  'KW',
  'EG',
  'MA',
  'DZ',
  'TN',
  'LB',
  'JO',
  'PS',
  'IN',
  'KZ',
  'MD

<a id='2-4'></a>
### 2.4 - Opcional - Límites de Velocidad de la API

*Nota*: Esta es una sección opcional.

Otro aspecto importante a tener en cuenta al trabajar con APIs es respecto a los límites de velocidad. La limitación de velocidad es un mecanismo utilizado por las APIs para controlar la cantidad de solicitudes que un cliente puede hacer dentro de un período de tiempo especificado. Ayuda a prevenir abusos o sobrecarga de la API limitando la frecuencia o volumen de solicitudes de un solo cliente. Así es como generalmente funciona la limitación de velocidad:

- Cuotas de Solicitud: Las APIs pueden aplicar un número máximo de solicitudes que un cliente puede hacer dentro de una ventana de tiempo dada, por ejemplo, 100 solicitudes por minuto.

- Ventanas de Tiempo: La ventana de tiempo especifica la duración durante la cual se mide la cuota de solicitudes. Por ejemplo, un límite de velocidad de 100 solicitudes por minuto significa que el cliente puede hacer hasta 100 solicitudes en cualquier período de 60 segundos.

- Respuesta a Exceder Límites: Cuando un cliente excede el límite de velocidad, la API generalmente responde con un código de error (como 429 Demasiadas Solicitudes) o un mensaje que indica que se ha excedido el límite de velocidad. Esto permite a los clientes ajustar su comportamiento en consecuencia, como implementando [exponential backoff](https://medium.com/bobble-engineering/how-does-exponential-backoff-work-90ef02401c65) y otras estrategias de reintento. (Ver [aquí](https://harish-bhattbhatt.medium.com/best-practices-for-retry-pattern-f29d47cd5117) o [aquí](https://aws.amazon.com/builders-library/timeouts-retries-and-backoff-with-jitter/)). 

- Encabezados de Límites de Velocidad: Las APIs pueden incluir encabezados en la respuesta para indicar el estado actual del límite de velocidad del cliente, como el número de solicitudes restantes hasta que se restablezca el límite o el momento en que se restablecerá el límite.

La limitación de velocidad ayuda a mantener la estabilidad y fiabilidad de las APIs asegurando un acceso justo a los recursos y protegiendo contra comportamientos abusivos o maliciosos. También permite a los proveedores de API asignar recursos de manera más efectiva y gestionar las cargas de tráfico de manera más eficiente.

También puedes ver más detalles sobre los límites de velocidad de la API Web de Spotify en la [documentación](https://developer.spotify.com/documentation/web-api/concepts/rate-limits). En particular, esta API no aplica un límite rígido para la cantidad de solicitudes realizadas, sino que funciona de manera dinámica basada en la cantidad de llamadas dentro de una ventana móvil de 30 segundos. Puedes encontrar algunos [blogs](https://medium.com/mendix/limiting-your-amount-of-calls-in-mendix-most-of-the-time-rest-835dde55b10e#:~:text=The%20Spotify%20API%20service%20has,for%2060%20requests%20per%20minute) donde se han realizado experimentos para identificar el número promedio de solicitudes por minuto.

A continuación se proporciona un código que realiza una prueba de rendimiento de las llamadas a la API; puedes jugar con el número de solicitudes y el intervalo de solicitud para ver el tiempo promedio de una solicitud. En caso de que realices demasiadas solicitudes y violes los límites de velocidad, obtendrás un código de estado 429.

*Nota*: Este código puede tardar unos minutos en ejecutarse.


In [28]:
import time

# Define the Spotify API endpoint
endpoint = 'https://api.spotify.com/v1/browse/new-releases'

headers = get_auth_header(access_token=token.get('access_token'))

# Define the number of requests to make
num_requests = 200

# Define the interval between requests (in seconds)
request_interval = 0.1  # Adjust as needed based on the API rate limit

# Store the timestamps of successful requests
success_timestamps = []

# Make repeated requests to the endpoint
for i in range(num_requests):
    # Make the request
    response = requests.get(url=endpoint, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        success_timestamps.append(time.time())
    else:        
        print(f'Request {i+1}: Failed with code {response.status_code}')
    
    # Wait for the specified interval before making the next request
    time.sleep(request_interval)

# Calculate the time between successful requests
if len(success_timestamps) > 1:
    time_gaps = [success_timestamps[i] - success_timestamps[i-1] for i in range(1, len(success_timestamps))]
    print(f'Average time between successful requests: {sum(time_gaps) / len(time_gaps):.2f} seconds')
else:
    print('At least two successful requests are needed to calculate the time between requests.')

Average time between successful requests: 0.57 seconds


In [29]:
import time
import requests

# Define el endpoint de la API de Spotify
endpoint = 'https://api.spotify.com/v1/browse/new-releases'

headers = get_auth_header(access_token=token.get('access_token'))  # Obtiene los encabezados de autorización

# Define el número de solicitudes a realizar
num_requests = 200

# Define el intervalo entre solicitudes (en segundos)
request_interval = 0.1  # Ajusta según sea necesario basado en el límite de tasa de la API

# Almacena las marcas de tiempo de las solicitudes exitosas
success_timestamps = []

# Realiza solicitudes repetidas al endpoint
for i in range(num_requests):
    # Realiza la solicitud
    response = requests.get(url=endpoint, headers=headers)
    
    # Verifica si la solicitud fue exitosa
    if response.status_code == 200:
        success_timestamps.append(time.time())  # Almacena la marca de tiempo del éxito
    else:        
        print(f'Solicitud {i+1}: Falló con el código {response.status_code}')
    
    # Espera el intervalo especificado antes de realizar la siguiente solicitud
    time.sleep(request_interval)
    
# Calcula el tiempo entre solicitudes exitosas
if len(success_timestamps) > 1:
    time_gaps = [success_timestamps[i] - success_timestamps[i-1] for i in range(1, len(success_timestamps))]
    print(f'Promedio de tiempo entre solicitudes exitosas: {sum(time_gaps) / len(time_gaps):.2f} segundos')
else:
    print('Se necesitan al menos dos solicitudes exitosas para calcular el tiempo entre solicitudes.')

Promedio de tiempo entre solicitudes exitosas: 0.57 segundos


<a id='3'></a>
## 3 - Pipeline por lotes

Ahora que has aprendido lo básico de trabajar con APIs, vamos a crear un pipeline que extraiga la información de las canciones de los álbumes lanzados recientemente. Para eso, usarás dos endpoints:
* El mismo [endpoint Obtener lanzamientos recientes](https://developer.spotify.com/documentation/web-api/reference/get-new-releases) que usaste en los ejercicios anteriores.
* El [endpoint Obtener canciones de un álbum](https://developer.spotify.com/documentation/web-api/reference/get-an-albums-tracks). Este endpoint te permite obtener información del catálogo de Spotify sobre las canciones de un álbum.


En la carpeta `src/`, se te proporcionan tres scripts (`authentication.py`, `endpoint.py` y `main.py`) que te permitirán realizar dicha extracción.
- El archivo `endpoint.py` contiene dos llamadas a API paginadas. La primera `get_paginated_new_releases` te permite obtener la lista de nuevos lanzamientos de álbumes usando la misma llamada paginada que usaste en la primera parte. La segunda `get_paginated_album_tracks` te permite obtener información del catálogo de Spotify sobre las pistas de un álbum usando el endpoint Obtener Pistas del Álbum.
- El archivo `authentication.py` contiene el script de la función `get_token` que devuelve un token de acceso.
- El archivo `main.py` llama a la primera llamada a API paginada para obtener los ids de los nuevos álbumes. Luego, para cada id de álbum, se realiza la segunda llamada a API paginada para extraer la información del catálogo de cada id de álbum.

En este momento, el código gestiona solicitudes paginadas pero no hemos tenido en cuenta que nuestro token de acceso tiene un tiempo limitado, por lo que si tus solicitudes en el pipeline duran más de 3600 segundos, puedes obtener un error con código de estado 401. Entonces, el primer paso es escribir una rutina que maneje la actualización del token en `get_paginated_new_releases`. Sigue las instrucciones para implementar esta rutina.


<a id='ex04'></a>
### Ejercicio 4

Abre el archivo ubicado en `src/endpoint.py`.

Busca la función `get_paginated_new_releases`. Crea una condición if sobre `response.status_code` y compárala con el valor 401. Esto significa que si el código de estado devuelto es 401 (No autorizado) realizarás los siguientes pasos:
- Usa el argumento `kwargs` que se pasa a la función `get_paginated_new_releases`; pásalo a la función `get_token` y asígnalo a una variable llamada `token_response`.
- Crea una condición interna en la que verificarás si la clave `"access_token"` está en el diccionario `token_response`. Si es verdadero, llamarás a la función `get_auth_header` con el token de acceso correspondiente y asignarás el resultado a la variable `headers`. Observa el uso de la palabra clave `continue` para asegurarte de que la solicitud se ejecute nuevamente.
- Si la condición sobre `"access_token"` es falsa, simplemente devuelve una lista vacía.

Guarda los cambios en el archivo `src/endpoint.py`.


Ahora que sabemos cómo actualizar el token en caso de que expire, es momento de continuar con el resto del código. Ahora que tenemos los nuevos lanzamientos de álbumes, la idea es extraer la información de las pistas que componen cada álbum.

Para obtener esta información, utilizarás el [endpoint Obtener Pistas del Álbum](https://developer.spotify.com/documentation/web-api/reference/get-an-albums-tracks). Tómate un momento para leer la documentación y entender cómo solicitar datos desde este endpoint en particular.

Abre el archivo en `src/main.py`. Allí verás después de la llamada a la función `get_paginated_new_releases` que estás extrayendo los IDs de los álbumes de la respuesta y guardándolos en la lista `albums_ids`. Esos IDs serán utilizados en la solicitud. Además, busca la siguiente constante:

- `URL_ALBUM_TRACKS`: La URL base para obtener información de un álbum en particular. Echa un vistazo a la documentación, puedes ver que tendrás que complementar esa URL con el ID del álbum y con la cadena `tracks` para completar el endpoint.

Esta información será pasada a la función `get_paginated_album_tracks` para construir el endpoint completo para la llamada a la API. En el próximo ejercicio, trabajarás en completar esta función en el archivo `src/endpoint.py`.


<a id='ex05'></a>
### Ejercicio 5


Vuelve al archivo `src/endpoint.py`. Busca el comentario `Exercise 5` y sigue las instrucciones para completar la función `get_paginated_album_tracks`.

1. La plantilla de la función para `get_paginated_album_tracks` ya te ha sido proporcionada. Lo primero que debes hacer es llamar a la función `get_auth_header` con el token de acceso y pasarla a una variable `headers`.
2. Crea el `requests_url` usando los parámetros `base_url` y `album_id`. Observa que los `tracks` se añaden al endpoint de la URL.
3. En el bucle `while`, realiza una solicitud GET usando el `request_url` y `headers` que creaste en el paso anterior. Asigna el resultado a `response`.
4. Implementa la misma rutina de actualización del token que antes para la función `get_paginated_new_releases`.
5. Después de la actualización del token, convierte el `response` a JSON usando el método `.json()`. Asignalo a `response_json`.
6. Extiende la lista `album_data` con el valor de `"items"` en `response_json`.
7. Actualiza `request_url` con el valor de `"next"` en `response_json`.

Guarda los cambios en el archivo `src/endpoint.py`.


<a id='ex06'></a>
### Ejercicio 6

Vuelve al archivo `src/main.py`. Busca el comentario `Exercise 6`. Dentro del bucle que itera a través de los `albums_ids`, hay una llamada a la función `get_paginated_album_tracks`. Sigue las instrucciones para definir los siguientes parámetros para la función dada:
- `base_url`: Usa la constante `URL_ALBUM_TRACKS` definida para ti.
- `access_token`: Asegúrate de pasar el `"access_token"` del objeto `token`.
- `album_id`.
- `get_token`: pasa la misma función `get_token`.
- Finalmente, pasa el diccionario `kwargs` definido al inicio de la función `main()`.

La respuesta de la función será asignada a la variable `album_data`.

Guarda los cambios en el archivo `src/main.py`.


Dentro del mismo ciclo for, la respuesta `album_data` se agrega al diccionario `album_items`, usando el id del álbum correspondiente como clave. Finalmente, después de iterar a través de todos los álbumes, puedes ver que el diccionario `album_items` se guarda en un archivo JSON en el entorno local. Echa un vistazo al formato del nombre del archivo, que tiene en cuenta la fecha y hora actuales para evitar colisiones con otros archivos.


Ejecute los siguientes comandos en la terminal para ejecutar el script `main.py`:


```bash
cd src
python main.py
```


*Notas*: Para abrir la terminal, haz clic en Terminal -> Nueva Terminal en el menú:

<img src="images/VSCodeCourseraTerminal.png"  width="600"/>


Una vez que el script haya terminado, deberías poder ver un archivo llamado `album_items_<DATETIME>.json` en la carpeta `src`.


<a id='4'></a>
## 4 - Opcional - SDK Spotipy

En varios casos, los desarrolladores de la API también proporcionan un Kit de Desarrollo de Software (SDK) para conectarse y realizar solicitudes a los diferentes puntos finales de la API sin la necesidad de crear el código desde cero. Para la API web de Spotify, desarrollaron el [SDK Spotipy](https://spotipy.readthedocs.io/en/2.22.1/) para hacerlo. Veamos un ejemplo de cómo funcionará para replicar la extracción de datos del punto final de lanzamientos de álbumes nuevos de manera paginada.


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
credentials = SpotifyClientCredentials(
        client_id=CLIENT_ID, client_secret=CLIENT_SECRET
    )

spotify = spotipy.Spotify(client_credentials_manager=credentials)

Puedes ver que el objeto `credentials` maneja el proceso de autenticación y contiene el token que se usará en solicitudes posteriores.

*Nota*: Por favor, ignora el mensaje de `DeprecationWarning` si ves un token de acceso en la salida.


In [ ]:
credentials.get_access_token()

Vamos a obtener datos de los nuevos lanzamientos de álbumes, como hiciste en el ejemplo anterior:


In [ ]:
limit = 20
response = spotify.new_releases(limit=limit)

También puedes navegar por estas respuestas. Si revisas la documentación del [`new_releases` método](https://spotipy.readthedocs.io/en/2.22.1/#spotipy.client.Spotify.new_releases), puedes ver que puedes especificar el parámetro `offset`, como hiciste anteriormente.


<a id='ex07'></a>
### Ejercicio 7

Vamos a realizar una solicitud paginada con el SDK. Luego podemos verificar los resultados de esta solicitud paginada contra el número total de elementos que vimos en los ejercicios anteriores, que sería 100. Sigue las instrucciones para terminar el código para realizar solicitudes paginadas.

1. Extiende la lista `album_data` con los `items` de la clave `albums` en la respuesta anterior.
2. Obtén el número `total` de elementos de la respuesta y asígnalo a la variable `total_albums_elements`.
3. Crea una lista de índices de desplazamiento. Como ya tienes los datos de la primera llamada, tu índice de desplazamiento inicial debe ser el valor de `limit` que usaste para hacer la primera solicitud. La lista debe terminar en `total_albums_elements` y el ritmo debe ser el mismo valor de `limit`. Guárdalo en `offset_idx`.
4. Comienza la paginación: itera sobre cada índice en `offset_idx`. En `response_page`, asigna la respuesta de la solicitud al método `new_releases` usando el índice de desplazamiento correspondiente y el límite.
5. Extiende nuevamente `album_data` con los `items` del álbum que obtienes en `response_page`.

Puedes inspeccionar visualmente los valores en `album_data` y asegurarte de que la longitud de la lista sea la misma que el número total de elementos que están disponibles para solicitar.


In [ ]:
def paginated_new_releases_sdk(limit: int=20) -> list:

    album_data = []
    ### START CODE HERE ### (~ 6 lines of code)
    response.spotify.(limit=limit)
    album_data.extend(response.get('albums').get('items'))
    total_albums_elements = response.get('albums').get('total')
    offset_idx = list(range(limit, total_albums_elements, limit))

    for idx in offset_idx:         
        response_page = spotify.new_releases(limit=limit, offset=idx)
        album_data.extend(response_page.get('albums').get('items'))
    ### END CODE HERE ###
    return album_data
    
album_data_sdk = paginated_new_releases_sdk()
album_data_sdk[0]

In [ ]:
len(album_data_sdk)

En este laboratorio aprendiste lo básico de ingerir datos desde la API. Trabajaste con el proceso de autenticación y paginación de manera manual así como usando un SDK de API.
